In [1]:
import pandas as pd
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras import optimizers
from keras import regularizers
from keras.optimizers import SGD
from keras.constraints import maxnorm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
import statsmodels.api as sm
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
df = pd.read_csv('lego_Sets.csv')

In [3]:
df.list_price = df.list_price.astype(float)
df['review_difficulty'] = df['review_difficulty'].astype("category")
df['review_difficulty'] = df['review_difficulty'].cat.reorder_categories(['Very Easy',
                                                               'Easy',
                                                               'Average',
                                                               'Challenging',
                                                               'Very Challenging'],
                                                                        ordered = True)
df['review_difficulty'] = df['review_difficulty'].cat.codes
df.theme_name = df.theme_name.astype("category")
df.ages = df.ages.astype("category")


In [4]:
df1 = df.copy().dropna()
df1 = df1.drop(['prod_desc', 
                'prod_id', 
                'prod_long_desc', 
                'set_name',
                'theme_name',
                'country'], 
                 axis=1)
df2 = pd.get_dummies(df1)

In [5]:
X, Y = df2.drop(['list_price'], axis=1), df2['list_price']

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(
                                                    X, 
                                                    Y, 
                                                    test_size=0.3, 
                                                    random_state=361)

In [7]:
def create_model(momentum = 0.0,
                activation = 'softplus', #git
                learn_rate = 0.1, #git
                dropout_rate = 0.0,
                weight_constraint = 1,
                neurons = 100, #git
                init = 'lecun_uniform', #git
                optimizer = 'adagrad'): #git

    # create model
    NN_model = Sequential()
    NN_model.add(Dense(128, 
                   kernel_initializer=init,
                   input_dim = X_train.shape[1], 
                   activation=activation))

    # The Hidden Layers :
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))
    NN_model.add(Dropout(dropout_rate)) 
    NN_model.add(Dense(neurons, 
                       kernel_initializer=init,
                       activation=activation,
                      kernel_constraint=maxnorm(weight_constraint)))

    # The Output Layer :
    NN_model.add(Dense(1, kernel_initializer=init,
                       activation='linear'))

    
    NN_model.compile(loss='mean_squared_error', 
                  optimizer=optimizer, 
                  metrics=['mean_absolute_error',
                           'mean_squared_error'])
    return NN_model


model = KerasRegressor(build_fn=create_model, 
                        epochs = 50, 
                        batch_size = 32,
                        verbose = 1)

In [ ]:
def grid_search(model = model):
# Use scikit-learn to grid search 
    activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
    learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
    dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    weight_constraint=[1, 2, 3, 4, 5]
    neurons = [30, 100, 128, 256]
    init = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    optimizer = [ 'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    ##############################################################
    # grid search epochs, batch size
    epochs = [50, 100] # add 50, 100, 150 etc
    batch_size = [10, 32, 64] # add 5, 10, 20, 40, 60, 80, 100 etc
    param_grid = dict(# epochs=epochs,
    #                  batch_size=batch_size,
#                      activation=activation,
    #                  momentum=momentum,
    #                  learn_rate=learn_rate,
                     dropout_rate=dropout_rate,
                     weight_constraint=weight_constraint,
#                      neurons=neurons,
#                      init=init,
#                      optimizer=optimizer
                     )
    ##############################################################
    grid = GridSearchCV(estimator=model, 
                        param_grid=param_grid, 
                        scoring=['r2'],
                        refit = 'r2',
                        n_jobs=1,
                        cv = 2)
    grid_result = grid.fit(X_train, Y_train) 
    ##############################################################
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_r2']
    stds = grid_result.cv_results_['std_test_r2']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
grid_search(model)

In [10]:
testing = create_model()
neural_pred = list(testing.predict(X_test)[:,0])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
neural_forecast = ({'Actual': Y_test, 'Predicted': neural_pred})

In [12]:
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, neural_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, neural_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, neural_pred)))

Mean Absolute Error: 62.588226409567135
Mean Squared Error: 9916.084338695577
Root Mean Squared Error: 99.57953775096355
